# Задание.

В одной из странных вселенных есть сферическая планета, на которой живут жуки трёх цветов (`желтый=0, синий=1, красный=2`). При этом радиус планеты не известен, но известно что расстояния в этой вселенной меряется метрикой Эвклида.
Файл `planet.csv` в каждой строчке содержит в первой колонке цвет жука и в последующих `1500` колонках координаты жука.

Жуки подчинены странным законам и в определённой координате может быть жук только определённого цвета.

От вас требуется сгенерировать файл `bugs.csv` в котором будет информация в таком же виде как в `planet.csv` про `100 желтых, 100 красных, 100 синих` жуков.

Координаты ваших жуков должны быть на поверхности планеты, при этом до ближайшего жука из файла `planet.csv`, равно как и до жуков из вашего файла не должно быть меньше чем `epsilon=1.2`.

In [21]:
import numpy as np
import pandas as pd

# Загружаем датасет
df = pd.read_csv("planet.zip")
colors = df.iloc[:, 0].values       # цвета
coordinates = df.iloc[:, 1:].values # координаты

# Разделяем жуков по цветам
bugs_by_color = {c: coordinates[colors == c] for c in [0, 1, 2]}

# Параметры
epsilon = 1.2 # минимаальное расстояние до соседа
new_n = 100   # новых жуков на 1 цвет

# Находим радиус планеты (предполагаем, что жуки ползают по ней равномерно)
center = np.mean(coordinates, axis=0)              # центр масс всех жуков
rad = np.linalg.norm(coordinates - center, axis=1) # расстояния (радиусы) всех точек до центра
radius = np.mean(rad)                              # средний радиус планеты (по всем жукам)
# Можно было бы взять радиус по одной точке, но средний — надёжнее :)

# Функция для генерации случайной точки на нашей гиперсфере
def random_point(center, radius, dim=1500):
    vec = np.random.normal(size=dim)  # случайный вектор в 1500-мерном пространстве
    vec /= np.linalg.norm(vec)        # нормируем чтобы он лежал на единичной гиперсфере
    return center + radius * vec      # масштабируем и смещаем точку на поверхность планеты

# Создаём новых жуков
new_bugs = []
for color in [0, 1, 2]:
    existing_bugs = bugs_by_color[color]
    new_color_bugs = []
    while len(new_color_bugs) < new_n:
        new_bug = random_point(center, radius)

        # Проверяем минимальное расстояние до всех существующих и новых жуков
        if all(np.linalg.norm(new_bug - bug) >= epsilon for bug in existing_bugs) \
           and all(np.linalg.norm(new_bug - bug) >= epsilon for bug in new_color_bugs):
            new_color_bugs.append(new_bug) # ура, жук нам подходит!

    # Добавляем цвет
    for bug in new_color_bugs:
        new_bugs.append([color] + list(bug))

# Сохраняем в файл с заголовками
columns = ["color"] + [f"dim_{i+1}" for i in range(coordinates.shape[1])]
new_df = pd.DataFrame(new_bugs, columns=columns)
new_df.to_csv("bugs.csv", index=False)